In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = "{:.4f}".format

df = pd.read_csv("ecommerce.csv", index_col= 0)
df.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,,
1,D,Flight,4,2,177,3,low,F,44,1233,1
2,F,Flight,4,5,216,2,low,M,59,3088,1
3,A,Flight,2,2,183,4,low,M,48,3374,1
4,B,Flight,3,3,176,4,medium,M,10,1177,1
5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [2]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Customer_care_calls,10999.0000,4.0545,1.1415,2.0000,3.0000,4.0000,5.0000,7.0000
Customer_rating,10999.0000,2.9905,1.4136,1.0000,2.0000,3.0000,4.0000,5.0000
Cost_of_the_Product,10999.0000,210.1968,48.0633,96.0000,169.0000,214.0000,251.0000,310.0000
Prior_purchases,10999.0000,3.5676,1.5229,2.0000,3.0000,3.0000,4.0000,10.0000
Discount_offered,10999.0000,13.3732,16.2055,1.0000,4.0000,7.0000,10.0000,65.0000
Weight_in_gms,10999.0000,3634.0167,1635.3773,1001.0000,1839.5000,4149.0000,5050.0000,7846.0000
Reached.on.Time_Y.N,10999.0000,0.5967,0.4906,0.0000,0.0000,1.0000,1.0000,1.0000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10999 entries, 1 to 10999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Warehouse_block      10999 non-null  object
 1   Mode_of_Shipment     10999 non-null  object
 2   Customer_care_calls  10999 non-null  int64 
 3   Customer_rating      10999 non-null  int64 
 4   Cost_of_the_Product  10999 non-null  int64 
 5   Prior_purchases      10999 non-null  int64 
 6   Product_importance   10999 non-null  object
 7   Gender               10999 non-null  object
 8   Discount_offered     10999 non-null  int64 
 9   Weight_in_gms        10999 non-null  int64 
 10  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 1.0+ MB


In [4]:
cat_df = df[df.describe(include='object').columns]

우선 category 변수 빼고 분석 진행

In [5]:
num_df = df[df.describe(exclude='object').columns]

In [6]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler = MinMaxScaler()
# tmp = num_df[['Cost_of_the_Product', 'Discount_offered', 'Weight_in_gms']]
# tmp_df = pd.DataFrame(scaler.fit_transform(tmp),columns = tmp.columns)

# for i in tmp.columns:
#     tmp[i] = pd.DataFrame(scaler.fit_transform(tmp[i].values.reshape(-1,1)), columns=[i])

# num_df[tmp.columns] = tmp

In [7]:
idx = num_df[num_df['Cost_of_the_Product'].isna()].index.values
df.loc[idx]

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,,


In [8]:
num_df[num_df['Cost_of_the_Product'].isna()]

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(num_df.drop('Reached.on.Time_Y.N', axis=1), num_df['Reached.on.Time_Y.N'],
                                                    test_size=0.2, random_state=23)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as XGB

lr = LogisticRegression()
rf = RandomForestClassifier(random_state=23)
gbm = GradientBoostingClassifier(random_state=23)
dt = DecisionTreeClassifier(random_state=23)
xgb = XGB.XGBClassifier(random_state=23)

lr.fit(x_train, y_train)
rf.fit(x_train, y_train)
gbm.fit(x_train, y_train)
dt.fit(x_train, y_train)
xgb.fit(x_train, y_train)

print(f"lr score: {lr.score(x_train,y_train)}")
print(f"rf score: {rf.score(x_train,y_train)}")
print(f"gbm score: {gbm.score(x_train,y_train)}")
print(f"dt score: {dt.score(x_train,y_train)}")
print(f"xgb score: {xgb.score(x_train,y_train)}")

lr_pred = lr.predict_proba(x_test)[:,1]
rf_pred = rf.predict_proba(x_test)[:,1]
gbm_pred = gbm.predict_proba(x_test)[:,1]
dt_pred = dt.predict_proba(x_test)[:,1]
xgb_pred = xgb.predict_proba(x_test)[:,1]

from sklearn.metrics import roc_auc_score

print(f"lr roc: {roc_auc_score(y_test, lr_pred)}")
print(f"rf roc: {roc_auc_score(y_test, rf_pred)}")
print(f"gbm roc: {roc_auc_score(y_test, gbm_pred)}")
print(f"dt roc: {roc_auc_score(y_test, dt_pred)}")
print(f"xgb roc: {roc_auc_score(y_test, xgb_pred)}")

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:54:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
lr score: 0.6365496079099898
rf score: 1.0
gbm score: 0.716672349130583
dt score: 1.0
xgb score: 0.8740765996135924
lr roc: 0.7090744408978477
rf roc: 0.7358787577104264
gbm roc: 0.7454135149282071
dt roc: 0.6188543238259454
xgb roc: 0.7357859588982515


In [11]:
cat_df

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
ID,,,,
1,D,Flight,low,F
2,F,Flight,low,M
3,A,Flight,low,M
4,B,Flight,medium,M
5,C,Flight,medium,F
...,...,...,...,...
10995,A,Ship,medium,F
10996,B,Ship,medium,F
10997,C,Ship,low,F


In [12]:
df['Reached.on.Time_Y.N'].mean()

0.5966906082371125

In [13]:
tmp = df.groupby(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']).mean()['Reached.on.Time_Y.N'].reset_index()
tmp

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender,Reached.on.Time_Y.N
0,A,Flight,high,F,0.6667
1,A,Flight,high,M,0.5000
2,A,Flight,low,F,0.5469
3,A,Flight,low,M,0.5625
4,A,Flight,medium,F,0.6167
5,A,Flight,medium,M,0.6290
6,A,Road,high,F,0.7500
7,A,Road,high,M,0.8667
8,A,Road,low,F,0.6111
9,A,Road,low,M,0.4795


In [14]:
tmp_mean = tmp['Reached.on.Time_Y.N'].mean()
# tmp['Reached.on.Time_Y.N'] = tmp['Reached.on.Time_Y.N'].apply(lambda x: 1 if x >= tmp_mean else 0)
tmp['Reached.on.Time_Y.N'] = tmp['Reached.on.Time_Y.N'].apply(lambda x: 1 if x >= tmp_mean else 0)

In [15]:
tmp.columns = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender', '카테고리확률']

In [16]:
len(df), len(df.columns)

(10999, 11)

In [17]:
df = df.merge(tmp, on=list(tmp.columns[:-1].values))

In [18]:
df['카테고리확률'].value_counts()

0    6772
1    4227
Name: 카테고리확률, dtype: int64

In [19]:
df['Gender'] = pd.factorize(df['Gender'])[0]

In [20]:
df['Product_importance'] = pd.factorize(df['Product_importance'])[0]

In [21]:
df['Mode_of_Shipment'].value_counts()

Ship      7462
Flight    1777
Road      1760
Name: Mode_of_Shipment, dtype: int64

In [22]:
df['Mode_of_Shipment'] = df['Mode_of_Shipment'].apply(lambda x: 1 if x=='Ship' else 2 if x == 'Flight' else 0)

In [23]:
tmp_df = df.copy()

In [24]:
dummy = pd.get_dummies(tmp_df['Warehouse_block'], drop_first=True)
tmp_df.drop('Warehouse_block', axis=1, inplace=True)

In [25]:
tmp_df = tmp_df.merge(dummy, on=tmp_df.index).drop('key_0', axis=1)
tmp_df.head()

,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,카테고리확률,B,C,D,F
0,2,4,2,177,3,0,0,44,1233,1,1,0,0,1,0
1,2,3,4,250,3,0,0,3,2371,1,1,0,0,1,0
2,2,4,2,136,3,0,0,65,1962,1,1,0,0,1,0
3,2,3,3,155,3,0,0,29,1712,1,1,0,0,1,0
4,2,3,3,268,3,0,0,33,3872,1,1,0,0,1,0


In [26]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(tmp_df, tmp_df['카테고리확률']):
    s_train = tmp_df.loc[tr_idx]
    s_test = tmp_df.loc[te_idx]

In [27]:
x_train, y_train = s_train.drop('Reached.on.Time_Y.N', axis=1), s_train['Reached.on.Time_Y.N']
x_test, y_test = s_test.drop('Reached.on.Time_Y.N', axis=1), s_test['Reached.on.Time_Y.N']

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as XGB

lr = LogisticRegression()
rf = RandomForestClassifier(random_state=23)
gbm = GradientBoostingClassifier(random_state=23)
dt = DecisionTreeClassifier(random_state=23)
xgb = XGB.XGBClassifier(random_state=23)
ada = AdaBoostClassifier(random_state=23)

lr.fit(x_train, y_train)
rf.fit(x_train, y_train)
gbm.fit(x_train, y_train)
dt.fit(x_train, y_train)
xgb.fit(x_train, y_train)
ada.fit(x_train, y_train)


print(f"lr score: {lr.score(x_train,y_train)}")
print(f"rf score: {rf.score(x_train,y_train)}")
print(f"gbm score: {gbm.score(x_train,y_train)}")
print(f"dt score: {dt.score(x_train,y_train)}")
print(f"xgb score: {xgb.score(x_train,y_train)}")
print(f"ada score: {ada.score(x_train,y_train)}")

lr_pred = lr.predict_proba(x_test)[:,1]
rf_pred = rf.predict_proba(x_test)[:,1]
gbm_pred = gbm.predict_proba(x_test)[:,1]
dt_pred = dt.predict_proba(x_test)[:,1]
xgb_pred = xgb.predict_proba(x_test)[:,1]
ada_pred = ada.predict_proba(x_test)[:,1]

from sklearn.metrics import roc_auc_score

print(f"lr roc: {roc_auc_score(y_test, lr_pred)}")
print(f"rf roc: {roc_auc_score(y_test, rf_pred)}")
print(f"gbm roc: {roc_auc_score(y_test, gbm_pred)}")
print(f"dt roc: {roc_auc_score(y_test, dt_pred)}")
print(f"xgb roc: {roc_auc_score(y_test, xgb_pred)}")
print(f"ada roc: {roc_auc_score(y_test, ada_pred)}")

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:54:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
lr score: 0.6432549153312876
rf score: 1.0
gbm score: 0.7217865666553017
dt score: 1.0
xgb score: 0.9095351744516422
ada score: 0.6924650528469144
lr roc: 0.7093787996020781
rf roc: 0.7303323097435614
gbm roc: 0.7448420401790649
dt roc: 0.6053975074610368
xgb roc: 0.7485483792970046
ada roc: 0.7400070465347629


In [29]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
tmp_df['Weight_in_gms'] = pd.DataFrame(scaler.fit_transform(tmp_df['Weight_in_gms'].values.reshape(-1,1)))
tmp_df['Cost_of_the_Product'] = pd.DataFrame(scaler.fit_transform(tmp_df['Cost_of_the_Product'].values.reshape(-1,1)))

In [30]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(tmp_df, tmp_df['카테고리확률']):
    s_train = tmp_df.loc[tr_idx]
    s_test = tmp_df.loc[te_idx]
    
x_train, y_train = s_train.drop('Reached.on.Time_Y.N', axis=1), s_train['Reached.on.Time_Y.N']
x_test, y_test = s_test.drop('Reached.on.Time_Y.N', axis=1), s_test['Reached.on.Time_Y.N']

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as XGB

lr = LogisticRegression()
rf = RandomForestClassifier(random_state=23)
gbm = GradientBoostingClassifier(random_state=23)
dt = DecisionTreeClassifier(random_state=23)
xgb = XGB.XGBClassifier(random_state=23)
ada = AdaBoostClassifier(random_state=23)

lr.fit(x_train, y_train)
rf.fit(x_train, y_train)
gbm.fit(x_train, y_train)
dt.fit(x_train, y_train)
xgb.fit(x_train, y_train)
ada.fit(x_train, y_train)


print(f"lr score: {lr.score(x_train,y_train)}")
print(f"rf score: {rf.score(x_train,y_train)}")
print(f"gbm score: {gbm.score(x_train,y_train)}")
print(f"dt score: {dt.score(x_train,y_train)}")
print(f"xgb score: {xgb.score(x_train,y_train)}")
print(f"ada score: {ada.score(x_train,y_train)}")

lr_pred = lr.predict_proba(x_test)[:,1]
rf_pred = rf.predict_proba(x_test)[:,1]
gbm_pred = gbm.predict_proba(x_test)[:,1]
dt_pred = dt.predict_proba(x_test)[:,1]
xgb_pred = xgb.predict_proba(x_test)[:,1]
ada_pred = ada.predict_proba(x_test)[:,1]

from sklearn.metrics import roc_auc_score

print(f"lr roc: {roc_auc_score(y_test, lr_pred)}")
print(f"rf roc: {roc_auc_score(y_test, rf_pred)}")
print(f"gbm roc: {roc_auc_score(y_test, gbm_pred)}")
print(f"dt roc: {roc_auc_score(y_test, dt_pred)}")
print(f"xgb roc: {roc_auc_score(y_test, xgb_pred)}")
print(f"ada roc: {roc_auc_score(y_test, ada_pred)}")

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:54:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
lr score: 0.645073303784521
rf score: 1.0
gbm score: 0.7217865666553017
dt score: 1.0
xgb score: 0.9095351744516422
ada score: 0.6924650528469144
lr roc: 0.7135134574997237
rf roc: 0.7303215154194761
gbm roc: 0.7448532662761136
dt roc: 0.6040711009174312
xgb roc: 0.7485803304962971
ada roc: 0.7400070465347629


In [32]:
df.corr()

,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,카테고리확률
Mode_of_Shipment,1.0000,0.0103,-0.0026,-0.0069,-0.0027,0.0077,0.0175,0.0013,-0.0035,0.0078,0.0410
Customer_care_calls,0.0103,1.0000,0.0122,0.3232,0.1808,-0.0581,0.0025,-0.1308,-0.2766,-0.0671,-0.0051
Customer_rating,-0.0026,0.0122,1.0000,0.0093,0.0132,0.0040,0.0028,-0.0031,-0.0019,0.0131,-0.0038
Cost_of_the_Product,-0.0069,0.3232,0.0093,1.0000,0.1237,-0.0468,0.0198,-0.1383,-0.1326,-0.0736,-0.0108
Prior_purchases,-0.0027,0.1808,0.0132,0.1237,1.0000,0.0273,-0.0094,-0.0828,-0.1682,-0.0555,0.0174
Product_importance,0.0077,-0.0581,0.0040,-0.0468,0.0273,1.0000,-0.0030,0.0260,0.0932,0.0205,0.2487
Gender,0.0175,0.0025,0.0028,0.0198,-0.0094,-0.0030,1.0000,-0.0118,0.0036,0.0047,0.0164
Discount_offered,0.0013,-0.1308,-0.0031,-0.1383,-0.0828,0.0260,-0.0118,1.0000,-0.3761,0.3971,0.0242
Weight_in_gms,-0.0035,-0.2766,-0.0019,-0.1326,-0.1682,0.0932,0.0036,-0.3761,1.0000,-0.2688,-0.0214
Reached.on.Time_Y.N,0.0078,-0.0671,0.0131,-0.0736,-0.0555,0.0205,0.0047,0.3971,-0.2688,1.0000,0.0677


In [33]:
df['temp'] = pd.cut(df['Cost_of_the_Product'], bins=5, labels=[1,2,3,4,5])

In [34]:
df

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,카테고리확률,temp
0,D,2,4,2,177,3,0,0,44,1233,1,1,2
1,D,2,3,4,250,3,0,0,3,2371,1,1,4
2,D,2,4,2,136,3,0,0,65,1962,1,1,1
3,D,2,3,3,155,3,0,0,29,1712,1,1,2
4,D,2,3,3,268,3,0,0,33,3872,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,C,0,3,5,223,7,2,1,8,4178,1,1,3
10995,C,0,3,1,242,10,2,1,9,5151,1,1,4
10996,C,0,5,1,258,3,2,1,3,4908,0,1,4
10997,C,0,3,5,182,3,2,1,5,4183,1,1,3


In [35]:
tmp_df = df.copy()
dummy = pd.get_dummies(tmp_df['Warehouse_block'], drop_first=True)
tmp_df.drop('Warehouse_block', axis=1, inplace=True)

In [38]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
tmp_df['Weight_in_gms'] = pd.DataFrame(scaler.fit_transform(tmp_df['Weight_in_gms'].values.reshape(-1,1)))
tmp_df['Cost_of_the_Product'] = pd.DataFrame(scaler.fit_transform(tmp_df['Cost_of_the_Product'].values.reshape(-1,1)))

In [39]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(tmp_df, tmp_df['temp']):
    s_train = tmp_df.loc[tr_idx]
    s_test = tmp_df.loc[te_idx]
    
s_train.drop('temp', axis=1, inplace=True)
s_test.drop('temp', axis=1, inplace=True)
x_train, y_train = s_train.drop('Reached.on.Time_Y.N', axis=1), s_train['Reached.on.Time_Y.N']
x_test, y_test = s_test.drop('Reached.on.Time_Y.N', axis=1), s_test['Reached.on.Time_Y.N']

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as XGB

lr = LogisticRegression()
rf = RandomForestClassifier(random_state=23)
gbm = GradientBoostingClassifier(random_state=23)
dt = DecisionTreeClassifier(random_state=23)
xgb = XGB.XGBClassifier(random_state=23)
ada = AdaBoostClassifier(random_state=23)

lr.fit(x_train, y_train)
rf.fit(x_train, y_train)
gbm.fit(x_train, y_train)
dt.fit(x_train, y_train)
xgb.fit(x_train, y_train)
ada.fit(x_train, y_train)


print(f"lr score: {lr.score(x_train,y_train)}")
print(f"rf score: {rf.score(x_train,y_train)}")
print(f"gbm score: {gbm.score(x_train,y_train)}")
print(f"dt score: {dt.score(x_train,y_train)}")
print(f"xgb score: {xgb.score(x_train,y_train)}")
print(f"ada score: {ada.score(x_train,y_train)}")

lr_pred = lr.predict_proba(x_test)[:,1]
rf_pred = rf.predict_proba(x_test)[:,1]
gbm_pred = gbm.predict_proba(x_test)[:,1]
dt_pred = dt.predict_proba(x_test)[:,1]
xgb_pred = xgb.predict_proba(x_test)[:,1]
ada_pred = ada.predict_proba(x_test)[:,1]

from sklearn.metrics import roc_auc_score

print(f"lr roc: {roc_auc_score(y_test, lr_pred)}")
print(f"rf roc: {roc_auc_score(y_test, rf_pred)}")
print(f"gbm roc: {roc_auc_score(y_test, gbm_pred)}")
print(f"dt roc: {roc_auc_score(y_test, dt_pred)}")
print(f"xgb roc: {roc_auc_score(y_test, xgb_pred)}")
print(f"ada roc: {roc_auc_score(y_test, ada_pred)}")

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:55:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
lr score: 0.6448460052278668
rf score: 1.0
gbm score: 0.719399931810433
dt score: 1.0
xgb score: 0.8938515740425048
ada score: 0.6880327309921582
lr roc: 0.7240982905982907
rf roc: 0.7417329059829061
gbm roc: 0.7415675213675215
dt roc: 0.6199572649572649
xgb roc: 0.7258213675213676
ada roc: 0.7508897435897436
